In [3]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, value
import pandas as pd



In [4]:

# Define parameters
products = ['ProductA', 'ProductB']
periods = ['Week1', 'Week2', 'Week3']
plants = ['Plant1', 'Plant2']

# Time-varying production cost per plant
production_cost = {
    'Plant1': {'ProductA': [5, 6, 5], 'ProductB': [6, 7, 6]},
    'Plant2': {'ProductA': [4, 5, 4], 'ProductB': [7, 6, 7]}
}

# Plant capacity per period
plant_capacity = {
    'Plant1': [200, 150, 250],
    'Plant2': [150, 200, 200]
}

# Demand and costs
demand = {'ProductA': [100, 120, 150], 'ProductB': [80, 90, 100]}
backorder_penalty = {'ProductA': 10, 'ProductB': 12}
inventory_cost = {'ProductA': 2, 'ProductB': 2}

# Build model
model = LpProblem("Advanced_Supply_Planning", LpMinimize)

# Decision variables
prod = LpVariable.dicts("Prod", (plants, products, periods), lowBound=0)
inventory = LpVariable.dicts("Inventory", (products, periods), lowBound=0)
backorder = LpVariable.dicts("Backorder", (products, periods), lowBound=0)

# Objective: total cost
model += lpSum(
    prod[pl][p][t] * production_cost[pl][p][i]
    for i, t in enumerate(periods)
    for pl in plants
    for p in products
) + lpSum(
    inventory[p][t] * inventory_cost[p]
    for p in products
    for t in periods
) + lpSum(
    backorder[p][t] * backorder_penalty[p]
    for p in products
    for t in periods
)

# Capacity constraints per plant and period
for i, t in enumerate(periods):
    for pl in plants:
        model += lpSum(prod[pl][p][t] for p in products) <= plant_capacity[pl][i], f"Capacity_{pl}_{t}"

# Flow balance constraints
for p in products:
    for i, t in enumerate(periods):
        produced = lpSum(prod[pl][p][t] for pl in plants)
        inv_prev = inventory[p][periods[i-1]] if i > 0 else 0
        bo_prev = backorder[p][periods[i-1]] if i > 0 else 0
        model += produced + inv_prev == demand[p][i] + inventory[p][t] + backorder[p][t] - bo_prev, f"Flow_{p}_{t}"

# Solve
model.solve()

# Output results
prod_results = []
for pl in plants:
    for p in products:
        for t in periods:
            qty = value(prod[pl][p][t])
            prod_results.append({
                'Plant': pl,
                'Product': p,
                'Period': t,
                'Production Qty': qty
            })

inv_results = []
for p in products:
    for t in periods:
        inv = value(inventory[p][t])
        bo = value(backorder[p][t])
        inv_results.append({
            'Product': p,
            'Period': t,
            'Inventory': inv,
            'Backorder': bo
        })

# Convert to DataFrames
prod_df = pd.DataFrame(prod_results)
inv_df = pd.DataFrame(inv_results)

# Display results
print("=== Production Plan ===")
print(prod_df.to_string(index=False))

print("\n=== Inventory and Backorders ===")
print(inv_df.to_string(index=False))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sgromme/source/sp_constraints/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/adeef94a3a304f339e7a60108648dacf-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/adeef94a3a304f339e7a60108648dacf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 86 RHS
At line 99 BOUNDS
At line 100 ENDATA
Problem MODEL has 12 rows, 24 columns and 44 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 12 (0) rows, 18 (-6) columns and 34 (-10) elements
0  Obj 0 Primal inf 639.99999 (6)
12  Obj 3230
Optimal - objective value 3230
After Postsolve, objective 3230, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 3230 - 12 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (